In [16]:
import sys
from mxnet import autograd, nd

In [17]:
def linreg(X, w, b):
    return nd.dot(X, w) + b

In [18]:
num_inputs = 2
num_examples = 1000
true_w =[2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))

print(features)
print(features[:, 0])

# labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels = linreg(features, nd.array(true_w).reshape(2,1), true_b)
labels = labels[:, 0]
labels += nd.random.normal(scale = 0.01, shape=(num_examples))
print(labels)


[[ 0.47742996  0.35282886]
 [ 0.81664836  0.9982429 ]
 [ 0.620633   -0.6347239 ]
 ...
 [-0.29119354 -1.3769919 ]
 [ 0.49655312  0.46195474]
 [ 0.22393152 -0.7298587 ]]
<NDArray 1000x2 @cpu(0)>

[ 4.77429956e-01  8.16648364e-01  6.20633006e-01 -1.24371326e+00
  1.81075048e+00 -8.13417852e-01  4.74758178e-01  9.91895497e-01
 -2.06755400e+00  6.41587734e-01  1.24029219e+00  4.02607955e-03
  3.69402438e-01 -9.03427124e-01 -1.74084771e+00 -2.54468709e-01
 -6.53186202e-01 -9.70252156e-01  1.58810830e+00 -6.85333192e-01
 -1.76222312e+00  4.14681286e-01  1.07255459e+00  4.91806008e-02
 -1.63072824e-01  9.15810615e-02 -1.68550000e-01  9.00005162e-01
 -3.17654681e+00  9.31486607e-01  7.03348100e-01 -1.29166320e-01
 -5.83155453e-01 -2.31245667e-01  1.83295643e+00 -4.91637081e-01
 -2.89979935e-01  1.22554302e+00 -5.58808088e-01 -6.23278737e-01
  4.41214412e-01 -4.27276909e-01 -1.10396004e+00  4.17290151e-01
 -8.21179003e-02 -6.18139982e-01 -5.24051249e-01  1.02053225e+00
 -6.34785295e-01 -1.49911

In [19]:
features[0], labels[0]

(
 [0.47742996 0.35282886]
 <NDArray 2 @cpu(0)>, 
 [3.9518158]
 <NDArray 1 @cpu(0)>)

In [20]:
from matplotlib import pyplot as plt
from IPython import display

def use_svg_display():
    # 用矢量图显示
    display.set_matplotlib_formats('svg')

def set_figsize(figsize=(4.5, 2.5)):
    use_svg_display()
    # 设置图的尺寸
    plt.rcParams['figure.figsize'] = figsize
    
set_figsize()
plt.scatter(features[:, 1].asnumpy(), labels.asnumpy(), 1);

In [21]:
import random
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)  # 样本的读取顺序是随机的
    for i in range(0, num_examples, batch_size):
        j = nd.array(indices[i: min(i + batch_size, num_examples)])
        yield features.take(j), labels.take(j)  # take函数根据索引返回对应元素
a = nd.array([[1,2,3], [4,5,6], [7,8,9]])
c = a.take(nd.array([0,1]))
print(c)


[[1. 2. 3.]
 [4. 5. 6.]]
<NDArray 2x3 @cpu(0)>


In [22]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, y)
    print(X[:])
    break
    


[[ 0.2606913   0.4742146 ]
 [ 0.6751286  -0.9932108 ]
 [-0.13900067 -0.65558493]
 [-0.88954407  0.9018987 ]
 [-1.8127162   0.12530951]
 [-1.0845165  -0.39076492]
 [-0.3154436  -0.84272766]
 [-1.9289646   1.757747  ]
 [-2.0001712   0.09219216]
 [ 0.35483548  0.5968538 ]]
<NDArray 10x2 @cpu(0)> 
[ 3.1035538   8.920315    6.1538625  -0.6352855   0.15106727  3.3678522
  6.4405017  -5.631573   -0.09039667  2.871418  ]
<NDArray 10 @cpu(0)>

[[ 0.2606913   0.4742146 ]
 [ 0.6751286  -0.9932108 ]
 [-0.13900067 -0.65558493]
 [-0.88954407  0.9018987 ]
 [-1.8127162   0.12530951]
 [-1.0845165  -0.39076492]
 [-0.3154436  -0.84272766]
 [-1.9289646   1.757747  ]
 [-2.0001712   0.09219216]
 [ 0.35483548  0.5968538 ]]
<NDArray 10x2 @cpu(0)>


## 1. Define model

In [23]:
net = linreg

## 2. Define loss function

In [24]:
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape))**2/2

## 3. Define Optimization algo

In [25]:
def sgd(params, lr, batch_size):
    for param in params:
        param[:] = param - lr*param.grad / batch_size #param[:] 表示解引用， 修改引用的那个object, 若改成param，则下面error不会收敛，因为w引用的object不会变，而param指向了新的object
        

## 4. MAIN

In [26]:
w = nd.random.normal(scale=0.01, shape=(num_inputs, 1))
b = nd.zeros(shape=(1,))
w, b
w.attach_grad()
b.attach_grad()

lr = 0.03
num_epochs = 5

loss = squared_loss  

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        with autograd.record():
            l = loss(net(X, w, b), y)
            l.backward()
        sgd([w,b], lr, batch_size)
    train_l = loss(net(features, w, b), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().asnumpy()))

epoch 1, loss 0.037517
epoch 2, loss 0.000141
epoch 3, loss 0.000050
epoch 4, loss 0.000050
epoch 5, loss 0.000050


In [64]:
true_w, w

([2, -3.4], 
 [[ 2.00067  ]
  [-3.3999734]]
 <NDArray 2x1 @cpu(0)>)

In [65]:
true_b, b

(4.2, 
 [4.199492]
 <NDArray 1 @cpu(0)>)